# **CMPT 726/419 A3 Q3: Optimization Algorithms**

---


**The test cases for this assignment are provided here. Don't mess with them!**
The assignment starts below.

In [1]:
import csv
import numpy as np
import torch
from tqdm import tqdm
import traceback

In [2]:
def generate_submission_hashes(optimizer, data_file, out_file):
    """Writes hashes of the result of running [optimizer] on data from
    [data_file] to [out_file]. After this is run, [out_file] can be submitted to
    Kaggle for evaluation.

    Args:
    optimizer   -- the optimizer being evaluated
    data_file   -- a .pt file wrapping a list where each item is a dictionary
                    as follows:
                    {
                        "hparams": dictionary of hyperparameters
                        "start_weights": array of starting weights
                        "gradients": list of gradients
                        "z": random vector
                    }
    out_file    -- file to write outputs to
    """
    data = torch.load(data_file)
    results = []

    for d in data:
        weights, hparams, state_info = d["start_weights"], d["hparams"], {}
        for g in d["gradients"]:
            weights, state_info = optimizer(weights, g, **state_info, **hparams)
        results.append(np.linalg.norm(weights - d["z"]))

    results = [[idx, r] for idx,r in enumerate(results)]
    with open(out_file, "w+") as f:
        writer = csv.writer(f)
        writer.writerow(["Index", "Hash"])
        for r in results:
            writer.writerow(r)

    tqdm.write(f"Wrote solution to {out_file}")

In [3]:
def equals_(x, y, tol=1e-4):
    """Returns if inputs [x] and [y] are equal, with equality defined as:
    1. [x] and [y] have identical hierarchical structure
    2. NumPy arrays in [x] and [y] must be equal up to some tolerance
    """
    if isinstance(x, np.ndarray) and isinstance(y, np.ndarray):
        if not x.shape == y.shape:
            return False
        else:
            cond1 = np.max(x / y) < 10 
            cond2 = np.min(x / y) > .1
            cond3 = np.mean(np.square(x - y)) < tol
            return cond1 and cond2 and cond3
    elif isinstance(x, dict) and isinstance(y, dict):
        if not x.keys() == y.keys():
            return False
        else:
            return all([equals_(x[k],y[k]) for k in x.keys()])
    elif isinstance(x, tuple) and isinstance(y, tuple):
        if not len(x) == len(y):
            return False
        else:
            return all([equals_(a,b) for a,b in zip(x,y)])
    elif isinstance(x, (int, float)) and isinstance(y, (int, float)):
        return  x / y > .1 and x / y < 10 and abs(x - y) < 1e-3
    else:
        return False

def test_optimizer(optimizer, weights, gradients, state_infos, hparams, results):
    """Prints test case information for [optimizer]. In the ith test, the ith
    elements of [weights], [gradients], [state_infos], and [hparams] are input
    to [optimizer], and the outputs of [optimizer] on these inputs are compared
    with the ith [element] of [results].

    Args:
    optimizer   -- an optimizer function
    weights     -- a list where each element is an array of weights
    gradients   -- a list where each element is an array of gradients
    state_infos -- a list where each element is a dictionary encoding the a 
                    state of the optimizer
    hparams     -- a list list of hyperparameters
    results     -- a list where each element is an array of updated weights
    """
    for w,g,s,h,r in zip(weights, gradients, state_infos, hparams, results):
        try:
            new_weights, new_state_info = optimizer(w, g, **s, **h)
            if equals_((new_weights, new_state_info), r):
                tqdm.write(f"=========== Test case passed :) ===========")
            else:
                tqdm.write(f"=========== Test case failed with incorrect output:( ===========")
                tqdm.write(f"Input weights:\n{w}\n")
                tqdm.write(f"Input gradients:\n{g}\n")
                tqdm.write(f"Input state information:\n{s}\n")
                tqdm.write(f"Input hyperparameters:{s}\n")
                tqdm.write(f"Expected new weights:{r[0]}\n")
                tqdm.write(f"Output new weights:{new_weights}\n")
                tqdm.write(f"Expected new state information:{r[1]}\n")
                tqdm.write(f"Output new state information:{new_state_info}\n")
        except Exception:
            tqdm.write(f"=========== Test case failed with error:( ===========")
            tqdm.write(f"Input weights:\n{w}\n")
            tqdm.write(f"Input gradients:\n{g}\n")
            tqdm.write(f"Input state information:\n{s}\n")
            tqdm.write(f"Input hyperparameters:{s}\n")
            traceback.print_exc()
    
    tqdm.write("")


In [4]:
def unzip(input): return zip(*input)


def test_sgd_with_momentum(optimizer):
    
    test_cases = [
        [np.array([[0.13184504,-0.2324543,0.151657,-0.3217226,0.2666803,]]),
        np.array([[-1.8109183,-1.4753062,-1.4077585,-2.0159426,-1.916701,]]),
        {'update': 0},
        {'lr': 1, 'mm': 0.9},
        (np.array([[1.9427633,1.2428519,1.5594155,1.69422,2.1833813,]]),
        {'update': np.array([[1.8109183,1.4753062,1.4077585,2.0159426,1.916701,]])})],

        [np.array([[1.9427633,1.2428519,1.5594155,1.69422,2.1833813,]]),
        np.array([[1.5052484,2.9311047,8.23906,7.965716,7.5918927,]]),
        {'update': np.array([[1.8109183,1.4753062,1.4077585,2.0159426,1.916701,]])},
        {'lr': 1, 'mm': 0.9},
        (np.array([[2.0673413,-0.36047733,-5.4126625,-4.4571476,-3.6834805,]]),
        {'update': np.array([[0.124578,-1.6033292,-6.972078,-6.1513677,-5.866862,]])})],

        [np.array([[2.0673413,-0.36047733,-5.4126625,-4.4571476,-3.6834805,]]),
        np.array([[-2.9772832,-7.9956026,-8.096151,-10.29695,-2.810056,]]),
        {'update': np.array([[0.124578,-1.6033292,-6.972078,-6.1513677,-5.866862,]])},
        {'lr': 1, 'mm': 0.9},
        (np.array([[5.156745,6.192129,-3.591381,0.30357218,-6.1536,]]),
        {'update': np.array([[3.0894034,6.5526066,1.8212814,4.76072,-2.4701197,]])})],

        [np.array([[-0.2871352,-0.02784033,0.23093075,0.41601038,-0.15191871,]]),
        np.array([[-1.9271317,-1.5067427,-0.19569726,-1.359275,-2.083497,]]),
        {'update': 0},
        {'lr': 2, 'mm': 0},
        (np.array([[3.5671282,2.985645,0.6223253,3.1345603,4.015075,]]),
        {'update': np.array([[3.8542633,3.0134854,0.39139453,2.71855,4.166994,]])})],

        [np.array([[3.5671282,2.985645,0.6223253,3.1345603,4.015075,]]),
        np.array([[11.763039,5.12741,1.0657036,7.0632687,4.160217,]]),
        {'update': np.array([[3.8542633,3.0134854,0.39139453,2.71855,4.166994,]])},
        {'lr': 2, 'mm': 0},
        (np.array([[-19.95895,-7.2691746,-1.509082,-10.991977,-4.3053584,]]),
        {'update': np.array([[-23.526077,-10.25482,-2.1314073,-14.126537,-8.320434,]])})],

        [np.array([[-19.95895,-7.2691746,-1.509082,-10.991977,-4.3053584,]]),
        np.array([[-17.727173,-46.071632,-28.641071,-26.136875,-21.374857,]]),
        {'update': np.array([[-23.526077,-10.25482,-2.1314073,-14.126537,-8.320434,]])},
        {'lr': 2, 'mm': 0},
        (np.array([[15.495396,84.87409,55.77306,41.281773,38.444355,]]),
        {'update': np.array([[35.454346,92.143265,57.282143,52.27375,42.749714,]])})],
    ]
    test_optimizer(optimizer, *unzip(test_cases))

def test_adam(optimizer):
    test_cases = [
        [np.array([[0.37969318,-0.10243413,-0.39066148,-0.23623134,-0.1174912,]]),
        np.array([[-0.25712273,-2.4049883,-0.8926169,-1.1141441,-2.6648488,]]),
        {'t': 0, 'm': 0, 'v': 0},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[0.47969317,-0.00243413,-0.29066148,-0.13623133,-0.0174912,]]),
        {'t': 1, 'm': np.array([[-0.02571227,-0.24049883,-0.0892617,-0.11141441,-0.2664849,]]), 'v': np.array([[6.6112094e-05,5.7839686e-03,7.9676503e-04,1.2413171e-03,7.1014194e-03,]])})],

        [np.array([[0.47969317,-0.00243413,-0.29066148,-0.13623135,-0.0174912,]]),
        np.array([[-0.8544274,-1.9550852,-1.49527,-0.74472547,-2.1240926,]]),
        {'t': 1, 'm': np.array([[-0.02571227,-0.24049883,-0.0892617,-0.11141441,-0.2664849,]]), 'v': np.array([[6.6112094e-05,5.7839686e-03,7.9676503e-04,1.2413171e-03,7.1014194e-03,]])},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[0.57025313,0.09650254,-0.19242549,-0.03916606,0.08129226,]]),
        {'t': 2, 'm': np.array([[-0.10858379,-0.41195744,-0.22986253,-0.17474551,-0.45224565,]]), 'v': np.array([[0.00079609,0.00960054,0.0030318,0.00179469,0.01160609,]])})],

        [np.array([[0.57025313,0.09650254,-0.19242549,-0.03916607,0.08129227,]]),
        np.array([[-0.2080417,-1.7005428,-0.2782645,-1.4111687,-0.94990975,]]),
        {'t': 2, 'm': np.array([[-0.10858379,-0.41195744,-0.22986253,-0.17474551,-0.45224565,]]), 'v': np.array([[0.00079609,0.00960054,0.0030318,0.00179469,0.01160609,]])},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[0.65293866,0.19428656,-0.10735527,0.05881965,0.17200929,]]),
        {'t': 3, 'm': np.array([[-0.11852959,-0.54081595,-0.2347027,-0.29838783,-0.5020121,]]), 'v': np.array([[0.00083858,0.01248279,0.0031062,0.00378429,0.01249681,]])})],

        [np.array([[-0.42728347,-0.18973799,0.23884566,0.38648924,0.22987218,]]),
        np.array([[-2.4048717,-2.3448138,-0.23648517,-0.7440031,-0.71357477,]]),
        {'t': 0, 'm': 0, 'v': 0},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[-0.32728347,-0.08973799,0.33884567,0.48648924,0.3298722,]]),
        {'t': 1, 'm': np.array([[-0.24048717,-0.23448138,-0.02364852,-0.07440031,-0.07135748,]]), 'v': np.array([[5.7834079e-03,5.4981522e-03,5.5925237e-05,5.5354065e-04,5.0918898e-04,]])})],

        [np.array([[-0.32728347,-0.089738,0.33884564,0.48648924,0.3298722,]]),
        np.array([[-0.34394962,-0.30774856,-0.319684,-0.51832855,-0.3830285,]]),
        {'t': 1, 'm': np.array([[-0.24048717,-0.23448138,-0.02364852,-0.07440031,-0.07135748,]]), 'v': np.array([[5.7834079e-03,5.4981522e-03,5.5925237e-05,5.5354065e-04,5.0918898e-04,]])},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[-0.25041252,-0.01361425,0.43851656,0.5840107,0.4241112,]]),
        {'t': 2, 'm': np.array([[-0.25083342,-0.2418081,-0.05325206,-0.11879314,-0.10252458,]]), 'v': np.array([[0.00589593,0.00558736,0.00015807,0.00082165,0.00065539,]])})],

        [np.array([[-0.25041252,-0.01361425,0.43851656,0.5840107,0.42411122,]]),
        np.array([[-0.01939912,-0.8510869,-0.1728474,-0.39118072,-0.16054946,]]),
        {'t': 2, 'm': np.array([[-0.25083342,-0.2418081,-0.05325206,-0.11879314,-0.10252458,]]), 'v': np.array([[0.00589593,0.00558736,0.00015807,0.00082165,0.00065539,]])},
        {'lr': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
        (np.array([[-0.19048236,0.06339748,0.5346486,0.67854196,0.507998,]]),
        {'t': 3, 'm': np.array([[-0.22768998,-0.30273598,-0.06521159,-0.14603189,-0.10832706,]]), 'v': np.array([[0.00589041,0.00630612,0.00018779,0.00097385,0.00068051,]])})],
    ]
    test_optimizer(optimizer, *unzip(test_cases))

# Directions
Don't edit anything above this cell.
See the PDF. Read it carefully. Good luck!

In [5]:
from IPython.core.display import update_display
def sgd_with_momentum(weights, gradient, lr=1, mm=.9, update=0):
    """Performs one step of gradient descent on weights [weights] with gradient
    [gradient] using SGD.

    Args:
    weights   (array): weights being optimized
    gradient  (array): gradients of [weights]
    lr        (float): the learning rate
    mm        (float): momentum parameter
    update    (array): the change in weights being optimized computed during the
                        previous iteration, or zero on the first iteration

    Returns:
    A (weights, dict) tuple where [weights] is an array of weights after one
    step of SGD, and [dict] is a dictionary with a key 'update' mapping to an
    array that is the change in [weights] computed during the optimizer step.
    """
    ##### YOUR CODE STARTS HERE ################################################
    update = mm * update - lr*gradient
    weights = weights + update

    ##### YOUR CODE ENDS HERE ##################################################
    assert isinstance(weights, np.ndarray)
    assert isinstance(update, np.ndarray)
    assert weights.shape == update.shape  
    return weights, {"update": update}

test_sgd_with_momentum(sgd_with_momentum)

=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========



In [14]:
def adam(weights, gradient, lr=1, t=0, m=0, v=0, beta1=.9, beta2=.999, epsilon=1e-8):
    """Performs one step of gradient descent on weights [weights] with gradient
    [gradient] using ADAM.

    Args:
    weights     (array): the weights being optimized from the prior iteration
    gradient    (array): gradient of [weights]
    lr          (float): the learning rate
    t           (int): index of prior iteration
    m           (array): first moment vector from prior iteration
    v           (array): second moment vector from prior iteration
    beta1       (float): exponential decay rate of first moment estimate
    beta2       (float): exponential decay rate of second moment estimate
    epsilon     (float): small number for preventing math errors

    Returns:
    A (weights, dict) tuple where [weights] is an array of weights after one
    step of SGD, and [dict] is a dictionary as follows:

    {
        "t": the index of the just-run iteration
        "m": the first moment vector for the next iteration
        "v": the second moment fector for the next iteration
    }
    """
    ##### YOUR CODE STARTS HERE ################################################

    t = t+1
    m = beta1*m + (1-beta1)*gradient
    v = beta2*v + (1-beta2)*(np.power(gradient, 2))
    temp_m = m/(1-np.power(beta1, t))
    temp_v = v/(1-np.power(beta2, t))
    sq = np.sqrt(temp_v)
    weights = weights - ((lr*temp_m)/(sq + epsilon))

    ##### YOUR CODE ENDS HERE ##################################################
    assert isinstance(t, int)
    assert isinstance(weights, np.ndarray)
    assert weights.shape == gradient.shape 
    assert isinstance(v, np.ndarray) and v.shape == weights.shape
    assert isinstance(m, np.ndarray) and m.shape == weights.shape
    return weights, {"t": t, "m": m, "v": v}

test_adam(adam)

=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========
=========== Test case passed :) ===========



In [17]:
generate_submission_hashes(adam, "adam_solution_generation_data.pt", "adam_hashes.csv")
generate_submission_hashes(sgd_with_momentum, "sgd_solution_generation_data.pt", "sgd_hashes.csv")

Wrote solution to adam_hashes.csv
Wrote solution to sgd_hashes.csv
